### this script sets a baseline for relation extraction using frequency-based BOW model

#### add additional features

In [114]:
import gzip
import numpy as np
import random
import os
import json

from collections import Counter, defaultdict, namedtuple
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import precision_recall_fscore_support, fbeta_score, make_scorer
from sklearn.pipeline import make_pipeline, Pipeline
from sklearn.model_selection import cross_val_score, StratifiedKFold, KFold
from sklearn.preprocessing import FunctionTransformer,LabelEncoder
import numpy as np
from sklearn.pipeline import FeatureUnion
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.feature_extraction import DictVectorizer

##### additional imports
import networkx as nx
import spacy
nlp = spacy.load('en')

In [3]:
##################################################################################################
# 1. LOAD DATA
##################################################################################################

PairExample = namedtuple('PairExample',
    'entity_1, entity_2, snippet')
Snippet = namedtuple('Snippet',
    'left, mention_1, middle, mention_2, right, direction')
def load_data(file, verbose=True):
    f = open(file,'r', encoding='utf-8')
    data = []
    labels = []
    for i,line in enumerate(f):
        instance = json.loads(line)
        if i==0:
            if verbose:
                print('json example:')
                print(instance)
        #'relation, entity_1, entity_2, snippet' fileds for each example
        #'left, mention_1, middle, mention_2, right, direction' for each snippet
        instance_tuple = PairExample(instance['entity_1'],instance['entity_2'],[])
        for snippet in instance['snippet']:
            try:
                snippet_tuple = Snippet(snippet['left'],snippet['mention_1'],snippet['middle'],
                                   snippet['mention_2'],snippet['right'],
                                    snippet['direction'])
                instance_tuple.snippet.append(snippet_tuple)
            except:
                print(instance)
        if i==0:
            if verbose:
                print('\nexample transformed as a named tuple:')
                print(instance_tuple)
        data.append(instance_tuple)
        labels.append(instance['relation'])
    return data,labels
    
train_data, train_labels = load_data('../data/train.json.txt')

json example:
{'relation': 'has_spouse', 'entity_1': 'Judy_Garland', 'entity_2': 'David_Rose', 'snippet': [{'left': 'thirty and his life and career were riding high . In 1941 , shortly after the death of his father , Mercer began an intense affair with nineteen-year-old', 'mention_1': 'Judy Garland', 'middle': 'while she was engaged to composer', 'mention_2': 'David Rose', 'right': '. Garland married Rose to temporarily stop the affair , but the effect on Mercer lingered , adding to the emotional depth of his lyrics . Their affair', 'direction': 'fwd'}]}

example transformed as a named tuple:
PairExample(entity_1='Judy_Garland', entity_2='David_Rose', snippet=[Snippet(left='thirty and his life and career were riding high . In 1941 , shortly after the death of his father , Mercer began an intense affair with nineteen-year-old', mention_1='Judy Garland', middle='while she was engaged to composer', mention_2='David Rose', right='. Garland married Rose to temporarily stop the affair , but 

In [4]:
# Statistics over relations
def print_stats(labels):
    labels_counts = Counter(labels)
    print('{:20s} {:>10s} {:>10s}'.format('', '', 'rel_examples'))
    print('{:20s} {:>10s} {:>10s}'.format('relation', 'examples', '/all_examples'))
    print('{:20s} {:>10s} {:>10s}'.format('--------', '--------', '-------'))
    for k,v in labels_counts.items():
        print('{:20s} {:10d} {:10.2f}'.format(k, v, v /len(labels)))
    print('{:20s} {:>10s} {:>10s}'.format('--------', '--------', '-------'))
    print('{:20s} {:10d} {:10.2f}'.format('Total', len(labels), len(labels) /len(labels)))

print('Train set statistics:')
print_stats(train_labels)

Train set statistics:
                                rel_examples
relation               examples /all_examples
--------               --------    -------
has_spouse                 3019       0.31
author                     2653       0.27
NO_REL                     2300       0.24
capital                     510       0.05
worked_at                  1178       0.12
--------               --------    -------
Total                      9660       1.00


In [5]:
# check that each entity pair is assigned only one relation
pair_dict={}
rel_dict={}
for example, label in zip(train_data,train_labels):
    if (example.entity_1,example.entity_2) not in pair_dict.keys():
        pair_dict[(example.entity_1,example.entity_2)] = [label]
        
    else:
        pair_dict[(example.entity_1,example.entity_2)].append(label)
        print(example.entity_1,example.entity_2,label)
    if label not in rel_dict.keys():
        rel_dict[label] = [example]
    else:
        rel_dict[label].append(example)
print("Done building dictionary")  
    
# example for each relation
for rel in rel_dict.keys():
    ex = rel_dict[rel][0]
    print(rel,ex.entity_1,ex.entity_2)

Done building dictionary
has_spouse Judy_Garland David_Rose
author Charlie_and_the_Chocolate_Factory Roald_Dahl
NO_REL Sichuan Tibet
capital Andalusia Seville
worked_at Carl-Henric_Svanberg Ericsson


In [6]:
# how to reconstruct full context

# ex = train_data[0]
# print(ex)
# print("\n full context:")
# s = ex.snippet[0]
# print(' '.join((s.left, s.mention_1, s.middle, s.mention_2, s.right)))

In [7]:
# def rebuild_text(ex):
#     rebuilt_ex = []
#     for s in ex.snippet:
#         text = ' '.join((s.left, s.mention_1, s.middle, s.mention_2, s.right))
#         rebuilt_ex.append(text)
#     return rebuilt_ex

In [8]:
# def build_text_from_snippet(s):
#     text = ' '.join((s.left, s.mention_1, s.middle, s.mention_2, s.right))
#     return text

In [9]:
# def rebuild_corpus(data):
#     corpus = []
#     for ex in data:
#         corpus.append(rebuild_text(ex)) 
#     return corpus

In [10]:
# def extract_key_sents(data):
#     key_sents = []
#     for ex in data:
#         m1 = ex.snippet[0].mention_1
#         m2 = ex.snippet[0].mention_2
#         text = build_text_from_snippet(ex.snippet[0])
#         doc = nlp(text)
#         for sent in doc.sents:
# #             print(sent)
#             if m1 in sent.string and m2 in sent.string:
#                 key_sents.append(sent)
#                 continue
                
#     return key_sents

In [11]:
# key_sents = extract_key_sents(train_data[:100])
# print(type(key_sents[0]))
# for sent in key_sents:
#     for chunk in sent.noun_chunks:
#         print(chunk.label_, chunk.text, chunk.root.text, chunk.root.dep_,
#           chunk.root.head.text)
#     for token in sent: 
#         print(token.text, token.dep_, token.head.text, token.head.pos_,
#               [child for child in token.children])

In [170]:
def tag_tokens(doc):
    tagged_ex = []
    
    for w in doc:
        if w.orth_ == "m_1" or w.orth_ == "m_2":
            tagged_ex.append(w.orth_)
        else:
            tagged_ex.append(w.pos_)
            
    tagged_ex = " ".join(tagged_ex)
    
    return tagged_ex

In [171]:
def lemmatize(doc):
    lemmas = []
    
    for w in doc:
        if w.lemma_ == "-PRON-" or w.orth_ == "m_1" or w.orth_ == "m_2":
            lemmas.append(w.orth_)
        else:
            lemmas.append(w.lemma_)
    
    lemmas = " ".join(lemmas)
    
    return lemmas

In [172]:
##################################################################################################
# 2.1 PERFORM NLP ON CORPUS DATA
##################################################################################################

def perform_nlp(data, verbose=True):
    
    if verbose:
        print("{} instances in data".format(len(data)))
        print("first instance looks like {}".format(data[0]))
        
    c = 0
    docs = []
    for instance in data:
        instance_context = []
        for s in instance.snippet:
            context = nlp(s.left + " m_1 " + s.middle + " m_2 " + s.right)
            instance_context.append(context)
        docs.append(instance_context)
        c += 1
    
        if verbose:
            if c % 1000 == 0:
                print("{} instances processed.".format(c))
        
    if verbose:
        print(len(docs))
        print(docs[0])
        print("Structure of context data is: {}-{}-{}".format(type(docs),
                                                              type(docs[0]),
                                                              type(docs[0][0])
                                                             )
             )
    
    return docs

In [173]:
def SelectContext(data, verbose=True):
    """BOW feature extraction"""
    only_context_data = []
    for instance in data:
        instance_context = []
        for s in instance.snippet:
            context = s.left + " m_1 " + s.middle + " m_2 " + s.right
            instance_context.append(context)
        only_context_data.append(' '.join(instance_context))
    if verbose:
        print(len(data))
        print(len(only_context_data))
        print(data[0])
        print(only_context_data[0])
    return only_context_data

In [174]:
def ExractSimpleFeatures(data, verbose=True):
    featurized_data = []
    for instance in data:
        featurized_instance = {'mid_words':'', 'distance':np.inf, 'mid_pos': ''}
        for s in instance.snippet:
            if len(s.middle.split()) < featurized_instance['distance']:
                featurized_instance['mid_words'] = s.middle
                featurized_instance['distance'] = len(s.middle.split())
                featurized_instance['mid_pos'] = ' '.join([w.pos_ for w in nlp(s.middle)])
        featurized_data.append(featurized_instance)
    if verbose:
        print(len(data))
        print(len(featurized_data))
        print(data[0])
        print(featurized_data[0])
        print(featurized_data[1])
    return featurized_data

In [175]:
def LengthOfEntities(data, verbose=True):
    featurized_data = []
    for instance in data:
        featurized_instance = {
            'entity1_len': len(instance.entity_1.split("_")),
            'entity2_len': len(instance.entity_2.split("_")),
            'combined_len': len(instance.entity_1.split("_")) + len(instance.entity_2.split("_"))
        }
        featurized_data.append(featurized_instance)
    if verbose:
        print(len(data))
        print(len(featurized_data))
        print(data[0])
        print(featurized_data[0])
        print(featurized_data[1])
    return featurized_data 

In [260]:
def FindDepPath(data, verbose=True):
    only_context_data = []
    for instance in data:
        instance_context = []
        for s in instance.snippet:
            context = s.left + " m_1 " + s.middle + " m_2 " + s.right
#             print(context)
            document = nlp(context)
            edges = []
            for token in document:
                # FYI https://spacy.io/docs/api/token
                for child in token.children:
                    edges.append(('{0}-{1}'.format(token.lower_,token.i),
                                  '{0}-{1}'.format(child.lower_,child.i)))

            graph = nx.Graph(edges)
#             print(graph)
            for w in graph:
#                 print(w)
                if "m_1" in w:
                    s = w
                if "m_2" in w:
                    t = w
            
#             print("s: ", len(s), "\t", "t: ", len(t))
#             if len(s) > 7 or len(t) > 7:
#                 pass
#             else:
            try:
                instance_context.append(nx.shortest_path_length(graph, source=s, target=t))
            except nx.NetworkXNoPath:
                instance_context.append(0)
            except nx.NodeNotFound:
                pass
        try:                                
            only_context_data.append(sum(instance_context)/len(instance_context))
        except ZeroDivisionError:
            only_context_data.append(0)
            
    if verbose:
        print(len(data))
        print(len(only_context_data))
        print(data[0])
        print(only_context_data[0])

    return only_context_data

In [263]:
# test_feat = LengthOfEntities(train_data)
# test_feat = ExractSimpleFeatures(train_data[:10])
# 
# test_feat = FindDepPath(train_data)

In [264]:
# print(len(test_feat))
# print(test_feat)

In [184]:
# def SelectTaggedContext(data, verbose=True):
    
#     processed_data = perform_nlp(data)
    
#     tagged_data = []
#     for processed_instance in processed_data:
#         instance_tags = []
#         for doc in processed_instance:  
#             tags = tag_tokens(doc)
#             instance_tags.append(tags)
#         tagged_data.append(' '.join(instance_tags))
#     if verbose:
#         print(len(processed_data))
#         print(len(tagged_data))
#         print(processed_data[0])
#         print(tagged_data[0])
#     return tagged_data

In [185]:
class SimpleFeaturizer(BaseEstimator, TransformerMixin):
    def __init__(self, featurizer):
        self.featurizers = featurizer

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        return ExractSimpleFeatures(X, verbose=False)

In [186]:
class EntityLengthFeaturizer(BaseEstimator, TransformerMixin):
    """Extract features from each isntance for DictVectorizer"""
    def __init__(self, featurizer):
        self.featurizers = featurizer
        
    def fit(self, x, y=None):
        return self

    def transform(self, X):
        return LengthOfEntities(X, verbose=False)

In [187]:
class BowFeaturizer(BaseEstimator, TransformerMixin):
    """BOW featurizer"""
    def __init__(self, featurizer):
        self.featurizers = featurizer
        
    def fit(self, x, y=None):
        return self

    def transform(self, X):
        return SelectContext(X, verbose=False)

In [192]:
class DependencyPath(BaseEstimator, TransformerMixin):
    """BOW featurizer"""
    def __init__(self, featurizer):
        self.featurizers = featurizer
        
    def fit(self, x, y=None):
        return self

    def transform(self, X):
        return FindDepPath(X, verbose=False)

In [193]:
# Transform labels to numeric values
le = LabelEncoder()
train_labels_featurized = le.fit_transform(train_labels)

# Fit model one vs rest logistic regression    
# clf = make_pipeline(DictVectorizer(), LogisticRegression())

# clf = make_pipeline(union, LogisticRegression())

# length_pipe = Pipeline([
#   ('ent_length', EntityLengthFeaturizer(LengthOfEntities)),
#   ('ent_length_vectorize', DictVectorizer())
#   ])

# bow_pipe = Pipeline([
#   ('context_data', BowFeaturizer(SelectContext)),
#   ('context_data_vectorize', CountVectorizer())
#   ])

# clf = Pipeline([
#     ('feat_union', FeatureUnion(transformer_list=[
#           ('length_pipeline', length_pipe),
#           ('bow_pipeline', bow_pipe)
#           ])),
#     ('classify', LogisticRegression())
#     ])


length_pipe = make_pipeline(EntityLengthFeaturizer(LengthOfEntities), DictVectorizer())

# bow_pipe = make_pipeline(BowFeaturizer(SelectContext), CountVectorizer(ngram_range=(1,3)))
bow_pipe = make_pipeline(BowFeaturizer(SelectContext), CountVectorizer())

simple_pipe = make_pipeline(SimpleFeaturizer(ExractSimpleFeatures), DictVectorizer())

# dep_pipe = make_pipeline(DependencyPath(FindDepPath))

clf = make_pipeline(FeatureUnion(transformer_list=[
    ('length_pipeline', length_pipe),
    ('bow_pipeline', bow_pipe),
    ('simple_pipeline', simple_pipe),
    ('dependency_pipeline', DependencyPath(FindDepPath))]),
    LogisticRegression())


In [190]:
##################################################################################################
# 3. TRAIN CLASSIFIER AND EVALUATE (CV)
##################################################################################################

def print_statistics_header():
    print('{:20s} {:>10s} {:>10s} {:>10s} {:>10s}'.format(
        'relation', 'precision', 'recall', 'f-score', 'support'))
    print('{:20s} {:>10s} {:>10s} {:>10s} {:>10s}'.format(
        '-' * 18, '-' * 9, '-' * 9, '-' * 9, '-' * 9))

def print_statistics_row(rel, result):
    print('{:20s} {:10.3f} {:10.3f} {:10.3f} {:10d}'.format(rel, *result))

def print_statistics_footer(avg_result):
    print('{:20s} {:>10s} {:>10s} {:>10s} {:>10s}'.format(
        '-' * 18, '-' * 9, '-' * 9, '-' * 9, '-' * 9))
    print('{:20s} {:10.3f} {:10.3f} {:10.3f} {:10d}'.format('macro-average', *avg_result))

def macro_average_results(results):
    avg_result = [np.average([r[i] for r in results.values()]) for i in range(3)]
    avg_result.append(np.sum([r[3] for r in results.values()]))
    return avg_result

def average_results(results):
    avg_result = [np.average([r[i] for r in results]) for i in range(3)]
    avg_result.append(np.sum([r[3] for r in results]))
    return avg_result
    
def evaluateCV(classifier, label_encoder, X, y, verbose=True):
    """
    classifier: clf - pipeline with CountVevtorizer and Logistic regression
    label_encoder: le - label encoder
    X: train data featurized
    y: train labels featurized
    """
    results = {}
    for rel in le.classes_:
#         print(rel)
        results[rel] = []
    if verbose:
        print_statistics_header()
        kfold = StratifiedKFold(n_splits = 5, shuffle=True, random_state=0) 
        for train_index, test_index in kfold.split(X, y):
            #print("TRAIN:", train_index, "TEST:", test_index)
            X_train, X_test = [X[i] for i in train_index], [X[i] for i in test_index]
            y_train, y_test = [y[i] for i in train_index], [y[i] for i in test_index]
            clf.fit(X_train, y_train)
            pred_labels = classifier.predict(X_test)
            stats = precision_recall_fscore_support(y_test, pred_labels, beta=0.5)
            #print(stats)
            for rel in label_encoder.classes_:
                rel_id = label_encoder.transform([rel])[0]
#             print(rel_id,rel)
                stats_rel = [stat[rel_id] for stat in stats]
                results[rel].append(stats_rel)
        for rel in label_encoder.classes_:
            results[rel] = average_results(results[rel])
            if verbose:
                print_statistics_row(rel, results[rel])
    avg_result = macro_average_results(results)
    if verbose:
        print_statistics_footer(avg_result)
    return avg_result[2]  # return f_0.5 score as summary statistic

In [191]:
evaluateCV(clf, le, train_data, train_labels_featurized)

relation              precision     recall    f-score    support
------------------    ---------  ---------  ---------  ---------
thirty-0
and-1
life-3
his-2
and-4
career-5
riding-7
were-6
high-8
.-9
in-10
1941-11
after-14
shortly-13
death-16
the-15
of-17
father-19
his-18
began-22
,-12
,-20
mercer-21
affair-25
engaged-36
.-40
an-23
intense-24
with-26
m_1-32
year-29
nineteen-27
--28
old-31
--30
while-33
she-34
was-35
to-37
m_2-39
composer-38
married-42
garland-41
rose-43
stop-46
,-49
but-50
lingered-55
to-44
temporarily-45
affair-48
the-47
effect-52
the-51
on-53
mercer-54
,-56
adding-57
.-65
to-58
depth-61
the-59
emotional-60
of-62
lyrics-64
his-63
affair-67
their-66
activity-1
latest-0
summary-8
wikia-7
sweet-4
--5
treat-6
4-2
other-3
of-9
and-15
factory-18
charlie-14
adventure-12
sweet-11
a-10
edit-13
the-16
chocolate-17
is-19
adaptation-23
.-30
a-20
2005-21
film-22
of-24
by-28
m_1-27
the-25
1964-26
m_2-29
directed-31
by-32
burton-34
tim-33
film-37
the-36
stars-38
,-35
highmore-40
as-

places-1
various-0
claim-2
title-4
.-25
the-3
,-5
as-7
such-6
parts-8
of-9
in-12
,-16
including-17
kham-11
southern-10
province-15
northwestern-13
yunnan-14
destinations-20
the-18
tourist-19
of-21
lijiang-22
and-23
zhongdian-24
places-26
like-27
m_1-28
and-29
m_2-30
claim-32
also-31
was-38
.-42
la-37
the-33
real-34
shangri-35
--36
in-39
territory-41
its-40
in-43
2001-44
region-48
tibet-46
autonomous-47
put-49
,-45
forward-50
proposal-52
la-61
a-51
optimise-57
regions-56
the-54
three-55
that-53
all-58
shangri-59
--60
research-1
more-0
.-2
taxonomists-4
some-3
group-5
nosed-8
snub-6
--7
monkeys-9
with-11
.-15
together-10
genus-14
the-12
pygathrix-13
nosed-18
snub-16
--17
monkeys-19
live-20
in-21
,-23
with-24
.-49
asia-22
range-26
a-25
covering-27
china-29
southern-28
(-30
m_1-32
especially-31
,-33
m_2-34
)-40
as-43
part-46
,-35
yunnan-36
,-37
and-38
guizhou-39
as-41
well-42
the-44
northern-45
of-47
vietnam-48
09/03/16-50
08:11:15-51
from-52
archive-61
radarchive-53
(-54
from-56
)-58
orig

themes-1
but-2
plot-6
a-3
simple-4
enough-5
and-7
humor-8
appeal-10
and-0
to-9
to-11
.-13
children-12
made-15
it-14
use-17
notable-16
of-18
music-20
popular-19
soundtrack-23
the-22
includes-24
;-21
music-25
.-51
by-26
covering-47
m_1-27
,-28
eels-29
,-30
m_2-31
,-32
proclaimers-34
the-33
,-35
wade-37
jason-36
,-38
men-41
the-39
baha-40
,-42
and-43
cale-45
john-44
(-46
cohen-49
)-50
leonard-48
during-52
june-53
2008-54
institute-59
,-55
the-56
american-57
film-58
gazes-0
at-1
.-10
enormity-3
the-2
,-4
detail-6
the-5
,-7
and-8
craftsmanship-9
inside-11
cathedral-13
the-12
of-14
málaga-15
tower-18
the-16
north-17
rises-19
meters-21
and-22
is-23
.-37
84-20
cathedral-28
highest-27
second-25
--26
the-24
in-29
behind-32
m_1-30
,-31
giralda-34
the-33
of-35
m_2-36
tower-40
the-38
south-39
is-41
unfinished-42
.-43
are-45
there-44
disputes-46
,-53
but-54
nicknamed-64
on-47
disappeared-51
funds-50
the-49
where-48
to-52
is-57
because-55
it-56
unfinished-58
cathedral-61
the-60
,-59
has-62
been-63
s-

alabama-0
,-1
and-2
treasoning-9
in-3
meantime-5
the-4
husband-7
her-6
is-8
her-10
with-11
.-18
star-14
a-12
porn-13
,-15
suxxx-17
candy-16
believed-21
this-19
is-20
be-23
and-36
habits-39
.-40
to-22
reference-25
m_2-35
a-24
to-26
m_1-27
,-28
wife-30
the-29
of-31
president-34
former-32
us-33
his-37
womanizing-38
•-42
v-41
auto-50
g-43
•-44
d-45
•-46
e-47
grand-48
theft-49
:-51
city-53
vice-52
characters-54
major-58
protagonist-55
tommy-56
vercetti-57
sang-1
and-0
``-2
clothes-6
''-7
,-8
mimicking-9
.-18
hand-5
second-3
--4
song-11
the-10
``-12
rose-16
''-17
hand-15
second-13
--14
skit-20
the-19
helped-21
restore-23
.-26
to-22
reputation-25
her-24
reagan-27
and-28
husband-30
in-44
.-46
her-29
with-31
as-34
predecessor-33
her-32
lady-36
first-35
,-37
m_1-38
,-43
,-39
widow-40
of-41
m_2-42
1985-45
nancy-47
and-48
jackie-49
compared-52
were-50
often-51
due-53
,-57
in-58
.-65
to-54
glamour-56
their-55
contrast-59
to-60
ladies-64
the-61
intervening-62
first-63
reagan-67
nancy-66
reflected-68

are-0
architects-2
.-6
the-1
of-3
process-5
that-4
in-7
couple-10
the-8
last-9
of-11
months-12
read-14
i-13
couple-16
book-31
and-39
enchantress-44
a-15
of-17
mention-20
books-18
that-19
palace-25
the-21
term-22
``-23
memory-24
''-26
m_1-28
's-29
history-30
--27
m_2-32
(-33
favorite-36
)-37
,-38
a-34
personal-35
rushdie-41
salman-40
's-42
the-43
of-45
.-47
florence-46
term-53
what-48
an-49
evocative-50
,-51
romantic-52
thought-56
,-54
i-55
.-57
called-59
also-58
method-61
,-65
a-66
``-60
of-62
''-64
loci-63
in-0
skies-2
,-6
and-7
in-8
by-15
.-22
the-1
(-3
1959-4
)-5
experience-11
the-9
mathematical-10
(-12
1981-13
)-14
davis-18
philip-16
j.-17
and-19
hersh-21
reuben-20
served-24
it-23
as-25
:-55
illustration-28
the-26
cover-27
for-29
,-36
account-39
,-45
for-46
1983-34
m_1-30
's-31
m_2-32
(-33
)-35
a-37
bestselling-38
of-40
history-42
the-41
of-43
science-44
matter-50
sorabji-48
richard-47
's-49
,-51
space-52
&-53
motion-54
theories-56
in-57
and-59
sequel-61
stephan-66
antiquity-58
the

...

show-0
,-1
wizards-2
be-8
of-3
place-5
waverly-4
,-6
would-7
last-10
.-11
its-9
in-12
february-13
2011-14
reported-17
,-15
deadline.com-16
acquired-25
.-37
pictures-20
universal-19
and-21
entertainment-23
strike-22
that-18
had-24
novel-27
by-29
with-31
the-26
m_1-28
m_2-30
gomez-32
attached-33
to-34
lead-36
the-35
month-40
that-38
same-39
guide-42
tv-41
reported-43
make-47
.-54
that-44
gomez-45
would-46
cameo-49
a-48
in-50
film-53
the-51
muppets-52
production-55
for-56
final-58
the-57
embarked-0
on-1
without-8
.-11
year-7
tour-5
mini-3
--4
a-6
the-2
tour-10
rain-9
in-12
february-13
2011-14
reported-17
,-15
deadline.com-16
acquired-25
.-37
pictures-20
universal-19
and-21
entertainment-23
strike-22
that-18
had-24
novel-27
by-29
with-31
the-26
m_1-28
m_2-30
gomez-32
attached-33
to-34
lead-36
the-35
39-39
[-38
]-40
month-43
that-41
same-42
guide-45
tv-44
reported-46
make-50
.-57
that-47
gomez-48
would-49
cameo-52
a-51
in-53
film-56
the-54
muppets-55
show-0
,-1
wizards-2
be-8
of-3
place-5
w

part-1
a-0
of-2
.-6
empire-5
the-3
russian-4
after-8
shortly-7
revolution-11
the-9
october-10
in-12
1917-13
declared-16
,-14
finland-15
independence-18
.-23
its-17
from-19
union-22
the-20
soviet-21
m_1-26
the-25
ceded-28
when-24
was-27
by-29
to-31
finland-30
m_2-33
the-32
after-34
war-37
the-35
winter-36
and-38
war-41
the-39
continuation-40
(-42
1939–1944-43
)-44
became-47
,-45
kuokkala-46
russian-48
.-49
railway-51
a-50
to-52
,-54
built-55
,-58
vyborg-53
in-56
1870-57
turned-59
repino-60
into-61
cottage-64
a-62
summer-63
wars-2
“-0
physics-1
”-3
.-4
known-8
well-6
--7
contribution-9
another-5
was-10
reality-12
.-29
quantum-11
by-13
herbert-16
physicist-14
nick-15
(-17
1985-18
)-19
dealt-21
which-20
mainly-22
with-23
interpretations-25
possible-24
of-26
theory-28
quantum-27
book-32
the-30
1979-31
,-33
m_1-34
”-45
)-46
by-35
physicist-44
m_2-36
“-41
confessedly-40
self-38
--39
(-37
not-42
a-43
included-48
again-47
parallels-49
.-56
between-50
mysticism-52
eastern-51
and-53
physics-55
mo

one-0
of-1
required-14
.-42
biggest-3
the-2
and-4
highlights-8
expensive-7
most-6
the-5
of-9
film-11
the-10
as-12
it-13
show-16
to-15
trains-20
the-17
five-18
running-19
shot-23
that-21
was-22
in-24
was-28
m_1-25
as-26
it-27
impossible-30
set-32
technically-29
to-31
it-33
up-34
in-35
m_2-36
or-37
city-41
any-38
other-39
big-40
trains-44
some-43
kept-46
were-45
on-47
for-49
on-54
.-58
standby-48
couple-51
a-50
of-52
days-53
tracks-57
two-55
railway-56
was-60
it-59
a-61
expensive-63
very-62
beach-1
the-0
see-5
,-2
one-3
can-4
ships-6
.-33
in-7
,-13
beyond-17
transit-8
along-9
coast-12
the-10
malabar-11
is-15
that-14
,-16
calicut-18
and-22
moving-23
(-19
kozhikode-20
)-21
towards-24
mangalore-25
,-26
m_1-27
m_2-31
and-28
bombay-29
(-30
)-32
laid-36
well-35
out-37
garden-38
the-34
and-39
sculpture-43
erected-48
the-40
massive-41
landscaped-42
of-44
mother-45
and-46
child-47
by-49
kunhiraman-53
sculptor-51
noted-50
kanayi-52
makes-54
captivating-57
.-58
it-55
extremely-56
interests-1
wide-0

is-1
name-0
“-2
reference-4
!-12
a-3
to-5
term-8
the-6
nautical-7
,-9
ahoy-11
ships-10
words-15
the-14
ahoy-18
!-19
“-16
chips-17
feature-22
”-20
also-21
prominently-23
in-24
.-36
story-26
a-25
appearing-27
,-33
by-34
in-28
chapter-29
15-30
of-31
m_1-32
m_2-35
1-38
[-37
]-39
.-40
relays-42
dickens-41
tale-45
a-43
childhood-44
of-46
shipwright-48
a-47
named-49
taunted-53
chips-50
who-51
is-52
by-54
rat-58
a-55
diabolical-56
talking-57
predicts-60
who-59
sinking-62
the-61
seat-0
in-1
at-23
.-29
centuries-8
11th-3
,-4
12th-5
and-6
13th-7
the-2
in-9
duchy-12
the-10
former-11
of-13
swabia-14
,-15
incorporated-17
,-22
which-16
aargau-21
day-20
present-18
--19
time-25
the-24
of-26
m_1-28
the-27
from-30
germany-32
southwestern-31
alsace-35
(-33
mainly-34
,-36
m_2-37
)-42
,-38
aargau-39
and-40
thurgau-41
family-44
the-43
extended-45
influence-47
to-50
,-59
austria-63
its-46
and-48
holdings-49
reaches-53
the-51
southeastern-52
of-54
empire-58
the-55
holy-56
roman-57
today-61
's-62
roughly-60
com

NodeNotFound: Source Snippet(left='RAFAEL of Israel , as well as a single universal vertical launcher capable of launching supersonic multi-role cruise missiles like the BrahMos and Sagarika . Prime Minister', mention_1='Manmohan Singh', middle='’ s wife', mention_2='Gursharan Kaur', right='will crack the auspicious coconut ( no , no Champagne bottles here in the Indian Navy ; - ) ) marking the launch of India ’ s first nuclear-powered ballistic missile', direction='fwd') not in G

In [37]:
# A check for the average F1 score

f_scorer = make_scorer(fbeta_score, beta=0.5, average='macro')

def evaluateCV_check(classifier, X, y, verbose=True):
    kfold = StratifiedKFold(n_splits = 5, shuffle=True, random_state=0) 
    scores = cross_val_score(classifier, X, y, cv=kfold, scoring = f_scorer)
    print("\nCross-validation scores (StratifiedKFold): ", scores)
    print("Mean cv score (StratifiedKFold): ", scores.mean())

In [38]:
evaluateCV_check(clf, train_data, train_labels_featurized)


Cross-validation scores (StratifiedKFold):  [0.78319178 0.77791582 0.78375181 0.78223485 0.78230726]
Mean cv score (StratifiedKFold):  0.7818803012927733


In [39]:
##################################################################################################
# 4. TEST PREDICTIONS and ANALYSIS
##################################################################################################

# Fit final model on the full train data
clf.fit(train_data, train_labels_featurized)

# Predict on test set
test_data, test_labels = load_data('../data/test-covered.json.txt', verbose=False)
print(len(test_labels))
# test_data_featurized = SelectContext(test_data, verbose=False)
test_label_predicted = clf.predict(test_data)
print(len(test_label_predicted))
# Deprecation warning explained: https://stackoverflow.com/questions/49545947/sklearn-deprecationwarning-truth-value-of-an-array
test_label_predicted_decoded = le.inverse_transform(test_label_predicted)
print(len(test_label_predicted_decoded))
print(test_label_predicted_decoded[:2])
f = open("outputs/test_labels.txt", 'w', encoding="utf-8")
for label in test_label_predicted_decoded:
    f.write(label+'\n')

1840
1840
1840
['capital' 'NO_REL']


/Applications/anaconda3/lib/python3.7/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


In [ ]:
# Feature analisys - print N most informative
# !! Make changes in this function when you change the pipleine!!
def printNMostInformative(classifier,label_encoder,N):
    """Prints features with the highest coefficient values, per class"""
    feature_names = classifier.named_steps['countvectorizer'].get_feature_names()

    coef = classifier.named_steps['logisticregression'].coef_    
    print(coef.shape)
    for rel in label_encoder.classes_:
        rel_id = label_encoder.transform([rel])[0]
        coef_rel = coef[rel_id]
        coefs_with_fns = sorted(zip(coef_rel, feature_names))
        top_features = coefs_with_fns[-N:]
        print("\nClass {} best: ".format(rel))
        for feat in top_features:
            print(feat)        
        
print("Top features used to predict: ")
# show the top features
printNMostInformative(clf,le,2)